# Quantification in HyperSpy

In [1]:
import hyperspy.api as hs

## Quantification with linear background removal

In [2]:
path="data/2022-09-06_EDS-SEM-APREO/"
file = "GaAs_30kV.emsa"
s = hs.load(path+file, signal_type='EDS_TEM')
s.add_elements(['As', 'Ga'])
# s.plot(xray_lines=True)

In [3]:
bw = s.estimate_background_windows(windows_width=2)
iw =  s.estimate_integration_windows(windows_width=2)
# s.plot(xray_lines=True, background_windows=bw, integration_windows=iw)

In [4]:
s_intensities = s.get_lines_intensity(background_windows=bw, integration_windows=iw)
k_factors = [4.191, 3.268] # As, Ga
quant = s.quantification(s_intensities, 'CL', factors=k_factors)
print(f'{file} quantification\nAs: {quant[0].data[0]:.2f} %, Ga: {quant[1].data[0]:.2f} %')

[########################################] | 100% Completed | 118.19 ms
GaAs_30kV.emsa quantification
As: 56.95 %, Ga: 43.05 %


## Quantification with intensity from model fit

In [5]:
# model fit
m = s.create_model(auto_background=False)
m.add_polynomial_background(order=6)
m.add_family_lines(['As_Ka', 'Ga_Ka'])

m.fit()
'm'
# m.plot(True)

'm'

In [6]:
m_intensities = m.get_lines_intensity()
for bs in m_intensities:
    print(bs.metadata.Sample.xray_lines)
k_factors = [4.191, 3.268] # As, Ga
m_intensities[0::2] # selects As Ka and Ga Ka
quant = s.quantification(m_intensities[0::2], 'CL', factors=k_factors)
print(f'{file} quantification with model fit')
print(f'As: {quant[0].data[0]:.2f} %, Ga: {quant[1].data[0]:.2f} %')

['As_Ka']
['As_La']
['Ga_Ka']
['Ga_La']
[########################################] | 100% Completed | 109.13 ms
GaAs_30kV.emsa quantification with model fit
As: 41.31 %, Ga: 58.69 %


## Quantification with intensity from model fit, after calibration

In [7]:
# quanitfication after calibration in HyperSpy

m.calibrate_energy_axis(calibrate='scale')
m.calibrate_energy_axis(calibrate='offset')

m_intensities = m.get_lines_intensity()
for bs in m_intensities:
    print(bs.metadata.Sample.xray_lines)

k_factors = [4.191, 3.268] # As, Ga
m_intensities[0::2] # selects As Ka and Ga Ka
quant = s.quantification(m_intensities[0::2], 'CL', factors=k_factors)
print(f'{file} quantification with model fit')
print(f'As: {quant[0].data[0]:.2f} %, Ga: {quant[1].data[0]:.2f} %')

# m.plot(True)


['As_Ka']
['As_La']
['Ga_Ka']
['Ga_La']
[########################################] | 100% Completed | 106.70 ms
GaAs_30kV.emsa quantification with model fit
As: 41.34 %, Ga: 58.66 %
